In [1]:
import numpy as np
import matplotlib.pylab as plt

In [2]:
def softmax(a):
    c = np.max(a)
    exp_a = np.exp(a-c)   #개선
    sum_exp_a = sum(np.exp(a-c))
    y = exp_a/sum_exp_a
    return y

In [3]:
def cross_entropy_error(y, t):
    if y.ndim == 1:
        t = t.reshape(1, t.size)
        y = y.reshape(1, y.size)
        
    # 훈련 데이터가 ONE-HOT 벡터라면 정답 레이블의 인덱스로 반환
    if t.size == y.size:
        t = t.argmax(axis=1)
             
    batch_size = y.shape[0]
    return -np.sum(np.log(y[np.arange(batch_size), t] + 1e-7)) / batch_size

In [4]:
class simpleNet:
    def __init__(self):
        np.random.seed(0) #항상 동일한 랜덤값 출력을 위함 (수업 목적)
        self.W = np.random.randn(2,3) # 정규분포로 초기화

    def predict(self, x):
        return np.dot(x, self.W)

    def loss(self, x, t):
        z = self.predict(x)
        y = softmax(z)
        loss = cross_entropy_error(y, t)

        return loss

In [5]:
net = simpleNet()

In [6]:
x = np.array([0.6, 0.9]) # 입력
t = np.array([0, 0, 1]) #정답, TARGET

In [7]:
net.W #어떤값으로 초기화 되어있는지 확인

array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])

### 입력값을 이용하여 FORWARD 방향 연산해보기

In [8]:
pred = net.predict(x)

In [9]:
print(pred)

[ 3.07523529  1.92089652 -0.2923073 ]


In [10]:
y = softmax(pred)

In [11]:
print(y)

[0.74088333 0.23357527 0.0255414 ]


In [12]:
np.argmax(y)

0

### 동일한 입력값과 정답을 이용하여 loss 구해보기

In [13]:
net.loss(x,t)

3.6674507891066104

### Gradient 구해보기

In [14]:
def numerical_gradient_single_point(f, x, verbose=False): 
    h = 1e-4
    grad = np.zeros_like(x)
    if verbose:
        print('x.size={}'.format(x.size)) # (x0, x1) 을 입력으로 받음 --> 2
       
    for idx in range(x.size): #축별로 계산
        v_keep = x[idx]
        
        # f(x+h) 계산
        x[idx] = float(v_keep) + h #n차원 입력 중 해당 차원으로만 h를 더하고
        fxh1 = f(x)
        if verbose:
            print(x, '-->', fxh1)
        
        # f(x-h) 계산
        x[idx] = float(v_keep) - h #n차원 입력 중 해당 차원으로만 h를 빼서
        fxh2 = f(x)
        if verbose:
            print(x, '-->', fxh2)
        
        grad[idx] = (fxh1 - fxh2) / (2*h) #n차원 방향의 차분을 구함 !
        x[idx] = v_keep # 값 복원
        
        if verbose:
            print('grad[{}]={}'.format(idx, grad[idx]))
            print()
    return grad

In [15]:
def numerical_gradient(f, X):
    if X.ndim == 1:
        return numerical_gradient_single_point(f, X)
    else:
        grad = np.zeros_like(X)
        
        for idx, x in enumerate(X):
            grad[idx] = numerical_gradient_single_point(f, x)
        
        return grad

In [16]:
def gradient_descent(f, init_x, lr =0.1, step_num= 100, return_history = False, verbose = False):
    x = init_x
    vx = np.zeros((step_num+1, 2)) # x의 궤적의 변화를 기록하기 위한 list 초기화
    vx[0] = x
    
    for i in range(step_num):
        grad = numerical_gradient(f, x)
        
        x -= lr * grad
        if verbose:
            print(x, grad)
            
        vx[i+1] = x # keep x in the history list
    
    if return_history:
        
        return x, vx
    else:
        return x

In [17]:
net.W

array([[ 1.76405235,  0.40015721,  0.97873798],
       [ 2.2408932 ,  1.86755799, -0.97727788]])

참고) lambda
```
lambda 인자리스트: 표현식
```

In [25]:
def f(w):
    return net.loss(x, t)

위처럼 해도 되지만, 아래처럼 lambda 를 써서 간단히 하는 것도 좋은 방법. 단, 인자로 들어가는 w는 dummy

In [26]:
f = lambda w: net.loss(x, t)

In [27]:
dW = numerical_gradient(f, net.W) # 6 방향의 기울기

In [28]:
print(dW)

[[ 0.44453517  0.14013839 -0.58467355]
 [ 0.66680275  0.21020758 -0.87701033]]


생각해보기
* 각 숫자들의 의미를 생각해보세요 (2020/11/24일 수업)
* 교재 135쪽 참조